**Step 1 : Setting up Spark Environment** 


**1.1 Deploy a Spark Cluster** (eg. AWS EMR, GCP Dataproc, Azure HD Insight)

**1.2 Store Data in HDFS**

Data source : [https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce](https://example.com)

**1.3 Use PySpark to interact with data.**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('OList Project').getOrCreate()

25/06/19 11:17:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
!hdfs dfs -ls /data/olist/

Found 9 items
-rw-r--r--   2 harishankargiri16 hadoop    9033957 2025-06-18 03:47 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop   61273883 2025-06-18 03:47 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop   15438671 2025-06-18 03:47 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop    5777138 2025-06-18 03:47 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop   14451670 2025-06-18 03:47 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop   17654914 2025-06-18 03:47 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop    2379446 2025-06-18 03:47 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop     174703 2025-06-18 03:47 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 harishankargiri16 hadoop       2613 2025-06-18 03:47 /data/olist/product_category_name_translation.c

In [3]:
customer_df = spark.read.csv('/data/olist/olist_customers_dataset.csv', header = True, inferSchema = True)
customer_df.show(5)
print(f'Customers : {customer_df.count()}')

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



Customers : 99441


In [5]:
customer_df = spark.read.csv('/data/olist/olist_customers_dataset.csv', header=True, inferSchema=True)
print(f'Customers : {customer_df.count()}')

geolocation_df = spark.read.csv('/data/olist/olist_geolocation_dataset.csv', header=True, inferSchema=True)
print(f'Geolocation : {geolocation_df.count()}')

order_items_df = spark.read.csv('/data/olist/olist_order_items_dataset.csv', header=True, inferSchema=True)
print(f'Order Items : {order_items_df.count()}')

order_payments_df = spark.read.csv('/data/olist/olist_order_payments_dataset.csv', header=True, inferSchema=True)
print(f'Order Payments : {order_payments_df.count()}')

order_reviews_df = spark.read.csv('/data/olist/olist_order_reviews_dataset.csv', header=True, inferSchema=True)
print(f'Order Reviews : {order_reviews_df.count()}')

orders_df = spark.read.csv('/data/olist/olist_orders_dataset.csv', header=True, inferSchema=True)
print(f'Orders : {orders_df.count()}')

products_df = spark.read.csv('/data/olist/olist_products_dataset.csv', header=True, inferSchema=True)
print(f'Products : {products_df.count()}')

sellers_df = spark.read.csv('/data/olist/olist_sellers_dataset.csv', header=True, inferSchema=True)
print(f'Sellers : {sellers_df.count()}')

Customers : 99441


Geolocation : 1000163


Order Items : 112650
Order Payments : 103886
Order Reviews : 104162
Orders : 99441
Products : 32951
Sellers : 3095


In [6]:
#check null values
from pyspark.sql.functions import col
for c in customer_df.columns:
    null_count = customer_df.filter(col(c).isNull()).count()
    print(f'{c} has {null_count} null values')

customer_id has 0 null values
customer_unique_id has 0 null values
customer_zip_code_prefix has 0 null values
customer_city has 0 null values
customer_state has 0 null values


In [7]:
#check duplicate values
customer_df.groupBy('customer_id').count().filter('count > 1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [40]:
customer_df.groupBy('customer_state').count().orderBy(col('count').desc()).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [41]:
orders_df.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [44]:
orders_df.groupBy('order_status').count().orderBy(col('count').asc()).show()

+------------+-----+
|order_status|count|
+------------+-----+
|    approved|    2|
|     created|    5|
|  processing|  301|
|    invoiced|  314|
| unavailable|  609|
|    canceled|  625|
|     shipped| 1107|
|   delivered|96478|
+------------+-----+



In [47]:
order_payments_df.groupBy('payment_type').count().orderBy(col('count').desc()).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [16]:
order_items_df.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

In [17]:
#Top selling item
from pyspark.sql.functions import sum
top_products = order_items_df.groupBy('product_id').agg(sum('price').alias('total_sales'))

In [9]:
top_products.orderBy(col('total_sales').desc()).show(10)

+--------------------+------------------+
|          product_id|       total_sales|
+--------------------+------------------+
|bb50f2e236e5eea01...|           63885.0|
|6cdd53843498f9289...| 54730.20000000005|
|d6160fb7873f18409...|48899.340000000004|
|d1c427060a0f73f6b...| 47214.51000000006|
|99a4788cb24856965...|43025.560000000085|
|3dd2a17168ec895c7...| 41082.60000000005|
|25c38557cf793876c...| 38907.32000000001|
|5f504b3a1c75b73d6...|37733.899999999994|
|53b36df67ebb7c415...| 37683.42000000001|
|aca2eb7d00ea1a7b8...| 37608.90000000007|
+--------------------+------------------+
only showing top 10 rows



In [22]:
#product delivery time
prod_delivery_df = orders_df.select('order_id', 'order_purchase_timestamp', 'order_delivered_customer_date')
prod_delivery_df.show(10)

+--------------------+------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|
+--------------------+------------------------+-----------------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|
|a4591c265e18cb1dc...|     2017-07-09 21:57:05|          2017-07-26 10:57:55|
|136cce7faa42fdb2c...|     2017-04-11 12:22:08|                         NULL|
|6514b8ad8028c9f2c...|     2017-05-16 13:10:30|          2017-05-26 12:55:51|
|76c6e866289321a7c...|     2017-01-23 18:29:09|          2017-02-02 14:08:10|
|e69bfb5eb88e0ed6a...|     2017-07-29 11:55:02|          2017-08

In [23]:
from pyspark.sql.functions import datediff
prod_delivery_df.withColumn('time_taken', datediff(col('order_delivered_customer_date'), col('order_purchase_timestamp'))).orderBy(col('time_taken').desc()).show()

+--------------------+------------------------+-----------------------------+----------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|time_taken|
+--------------------+------------------------+-----------------------------+----------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|       210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|       208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|       196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|       195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|       195|
|0f4519c5f1c541dde...|     2017-03-09 13:26:57|          2017-09-19 14:38:21|       194|
|47b40429ed8cce3ae...|     2018-01-03 09:44:01|          2018-07-13 20:51:31|       191|
|2fe324febf907e3ea...|     2017-03-13 20:17:10|          2017-09-19 17:00:07|       190|
|2d7561026d542c8db...

In [24]:
spark.stop()